In [ ]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

# Used to condense a question and chat history into a single question
condense_question_prompt_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language. If there is no chat history, just rephrase the question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(
    condense_question_prompt_template
)

# RAG Prompt to provide the context and question for LLM to answer
# We also ask the LLM to cite the source of the passage it is answering from
llm_context_prompt_template = """
Use the following passages to answer the user's question.
Each passage has a SOURCE which is the title of the document. When answering, cite source name of the passages you are answering from below the answer in a unique bullet point list.

If you don't know the answer, just say that you don't know, don't try to make up an answer.

----
{context}
----
Question: {question}
"""  # noqa: E501

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(llm_context_prompt_template)

# Used to build a context window from passages retrieved
document_prompt_template = """
---
NAME: {name}
PASSAGE:
{page_content}
---
"""

DOCUMENT_PROMPT = PromptTemplate.from_template(document_prompt_template)


In [ ]:
import os
from operator import itemgetter
from typing import List, Tuple

from langchain.retrievers import SelfQueryRetriever
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import format_document
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_elasticsearch.vectorstores import ElasticsearchStore

# from .prompts import CONDENSE_QUESTION_PROMPT, DOCUMENT_PROMPT, LLM_CONTEXT_PROMPT

ELASTIC_CLOUD_ID = os.getenv("ELASTIC_CLOUD_ID")
ELASTIC_USERNAME = os.getenv("ELASTIC_USERNAME", "elastic")
ELASTIC_PASSWORD = os.getenv("ELASTIC_PASSWORD")
ES_URL = os.getenv("ES_URL", "http://localhost:9200")
ELASTIC_INDEX_NAME = os.getenv("ELASTIC_INDEX_NAME", "workspace-search-example")

if ELASTIC_CLOUD_ID and ELASTIC_USERNAME and ELASTIC_PASSWORD:
    es_connection_details = {
        "es_cloud_id": ELASTIC_CLOUD_ID,
        "es_user": ELASTIC_USERNAME,
        "es_password": ELASTIC_PASSWORD,
    }
else:
    es_connection_details = {"es_url": ES_URL}

vecstore = ElasticsearchStore(
    ELASTIC_INDEX_NAME,
    embedding=OpenAIEmbeddings(),
    **es_connection_details,
)
# 🔤 中文: 工作场所政策的目的和规格。
document_contents = "The purpose and specifications of a workplace policy."
metadata_field_info = [
    {"name": "name", "type": "string", "description": "Name of the workplace policy."},
    {
        "name": "created_on",
        "type": "date",
        "description": "The date the policy was created in ISO 8601 date format (YYYY-MM-DD).",  # noqa: E501
    },
    {
        "name": "updated_at",
        "type": "date",
        "description": "The date the policy was last updated in ISO 8601 date format (YYYY-MM-DD).",  # noqa: E501
    },
    {
        "name": "location",
        "type": "string",
        "description": "Where the policy text is stored. The only valid values are ['github', 'sharepoint'].",
        # noqa: E501
    },
]
llm = ChatOpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vecstore, document_contents, metadata_field_info
)


def _combine_documents(docs: List) -> str:
    return "\n\n".join(format_document(doc, prompt=DOCUMENT_PROMPT) for doc in docs)


def _format_chat_history(chat_history: List[Tuple]) -> str:
    return "\n".join(f"Human: {human}\nAssistant: {ai}" for human, ai in chat_history)


class InputType(BaseModel):
    question: str
    chat_history: List[Tuple[str, str]] = Field(default_factory=list)


standalone_question = (
        {
            # 获取用户问题的参数，并做简单处理
            "question": itemgetter("question"),
            "chat_history": lambda x: _format_chat_history(x["chat_history"]),
        }
        # 将这两个字段组成的字典，传递给prompt，格式化成 prompt
        | CONDENSE_QUESTION_PROMPT
        # 把prompt交给llm作为输入
        | llm
        # 最后把llm的输入格式化
        | StrOutputParser()
)


def route_question(input):
    if input.get("chat_history"):
        return standalone_question
    else:
        return RunnablePassthrough()


## 先执行retriever，输入是问题，输出是文档list
## 再执行_combine_documents，输入是文档list，输出是str
## 再执行LLM_CONTEXT_PROMPT，输入 
# 添加额外参数，生成字典
_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

# standalone_question，作用是判断，是独立问题，还是具有记忆的问题
chain = (
    # 处理问题并生成新的问题
        standalone_question
        # 构建prompt所需的字典参数
        | _context
        # 结合上一步的字典，生成最终的prompt
        | LLM_CONTEXT_PROMPT
        # 结合上一步的prompt，生成最终的回答
        | llm
        # 最后格式化输出
        | StrOutputParser()
).with_types(input_type=InputType)  # 指定输入的参数类型
